#### Friday, November 8, 2024

[How to use chat models to call tools](https://python.langchain.com/docs/how_to/tool_calling/) plus [How to pass tool outputs to chat models](https://python.langchain.com/docs/how_to/tool_results_pass_to_model/)

mamba activate langchain

This runs without errors but with unexpected results.

Hmm I tried 3 different local models. 'mistral-7b-instruct-v0.3', 'hermes-3-llama-3.1-8b', and 'hugging-quants/llama-3.2-3b-instruct'. They all performed the same. The LMStudio logging showed all were able to perform the math functions, but langchain did not correctly parse the output.

In [39]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo") # "gpt-4o-mini")

llm = ChatOpenAI(base_url="http://localhost:1234/v1", 
                   model = "hermes-3-llama-3.1-8b",  # do not pass in an unrecognized model name ... 
                   api_key="lm-studio", 
                   temperature=0)

#### Python Functions

In [40]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

#### Pydantic class

In [41]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

#### Typedict class

In [42]:
from typing_extensions import Annotated, TypedDict


class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]

In [43]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content=' The product of 3 and 12 is 36.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 14, 'total_tokens': 28, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistral-7b-instruct-v0.3', 'system_fingerprint': 'mistral-7b-instruct-v0.3', 'finish_reason': 'stop', 'logprobs': None}, id='run-84076a09-3ac9-4b29-8401-50351506d889-0', usage_metadata={'input_tokens': 14, 'output_tokens': 14, 'total_tokens': 28})

#### Tool calls

In [44]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[]

#### Parsing

In [45]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[]

The next few cells are from a follow up document called [How to pass tool outputs to chat models](https://python.langchain.com/docs/how_to/tool_results_pass_to_model/)

I append the code into this document just to keep the notes in one notebook.

In [46]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [47]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[]


In [48]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content=' The product of 3 and 12 is 36. And the sum of 11 and 49 is 60.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 26, 'total_tokens': 56, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistral-7b-instruct-v0.3', 'system_fingerprint': 'mistral-7b-instruct-v0.3', 'finish_reason': 'stop', 'logprobs': None}, id='run-408367b5-04b6-4d72-acbf-b761f2f43c76-0', usage_metadata={'input_tokens': 26, 'output_tokens': 30, 'total_tokens': 56})]

In [49]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 0, 'prompt_tokens': 57, 'total_tokens': 57, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistral-7b-instruct-v0.3', 'system_fingerprint': 'mistral-7b-instruct-v0.3', 'finish_reason': 'stop', 'logprobs': None}, id='run-c57f9cfe-d141-49af-b55e-2b9cfa8103fb-0', usage_metadata={'input_tokens': 57, 'output_tokens': 0, 'total_tokens': 57})